# Setup and Preprocessing

## Drive Setup

In [ ]:
'''
Google Drive Setup
'''
from google.colab import drive
drive.mount('/content/drive')
'''
Move to required directory
'''
%ls
%cd drive/
%cd MyDrive/
%cd 'Thesis'/
%ls

## Imports

In [ ]:
'''
Importing required packages
'''

import pandas as pd

import re

import math

from datetime import datetime
from datetime import date

import numpy as np
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import tensorflow as tf
# from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense , Input , LSTM , SimpleRNN , Embedding, Dropout , Activation, Flatten , Conv1D ,Concatenate
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, SpatialDropout1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

## Loading Data

In [ ]:
'''
Reading in Data
'''
df = pd.read_excel(r'Data/CombinedData.xlsx')
df.head()

In [5]:
'''
Extracting data for preprocessing
'''
genre = df.iloc[:,22]
artist = df.iloc[:,21]
abstract = df.iloc[:,20]

releaseDate = df.iloc[:,24]

preprocessedData = df.iloc[:,4:20]

In [ ]:
#Showing the first elements of the genre variable
genre.head()

In [ ]:
#Showing the first elements of the artist variable
artist.head()

In [ ]:
#Showing the first elements of the abstract variable
abstract.head()

In [ ]:
#Showing the first elements of the release date variable
releaseDate.head()

In [ ]:
#Showing the first elements of the preprocessed data variable
preprocessedData.head()

## Text Preprocessing

In [11]:
'''
Function to preprocess text - 
i)    creats final string containing artist , genre and abstract
ii)   chnages string to lower 
iii)  removes punctuations
iv)   adds <bof> to start of string and <eos> to end
'''
def preprocess(val):
    message = 'artist is ' + str(artist[val]) + ' , genre is ' + genre[val] + ' , ' + abstract[val]
    # Lowercase the message
    text = message.lower()
    # Replace everything not a letter or apostrophe with a space
    text = re.sub('[^a-zA-Z\'0-9,&]+', ' ', text)
    text = '<bof>' + text + '<eos>'
    return text

In [12]:
#Running above function 
preprocessedText = []

for x in range(len(df)):
  preprocessedText.append(preprocess(x))

In [ ]:
#Showing the first elements of the output for preprocess function
preprocessedText[1:5]

## Date processing

In [14]:
'''
Gets Days between current date and release date of song
'''
today = date.today()

def days_between(d):
    rd = releaseDate[d]
    rd = rd.date()
    daysReleased = today - rd
    return daysReleased.days

In [15]:
#Running above function
daysSinceRelease = [] 
for d in range(len(df)):
  daysSinceRelease.append(days_between(d))

In [ ]:
#Showing the first elements of the output for days_between function
daysSinceRelease[1:5]

## Dataset

In [17]:
'''
Creating the final dataset
'''
preprocessedData['Days Released'] = daysSinceRelease
preprocessedData['Text'] = preprocessedText

In [ ]:
preprocessedData.head()

## Splitting Data

In [ ]:
#Splitting the data into test and train
train_Data, test_Data = train_test_split(preprocessedData, test_size=0.2, random_state=1)


print('Length of train X data: '+ str(len(train_Data)))
print('Length of test X data: '+ str(len(test_Data,)))

In [ ]:
#Showing head of train dataset
train_Data.head()

In [ ]:
#Showing head of test dataset
test_Data.head()

## Tokenize

In [ ]:
'''
tokenizing text data
'''

#variables to hold data
tokensList = []
max_sequence = 0

#tokenising sentences and getting max sequence

all_CulturalData = preprocessedData["Text"]

for sent in all_CulturalData: 
  tokens = word_tokenize(sent)
  sequence = len(tokens)
  tokensList.extend(tokens)
  if sequence > max_sequence:
    max_sequence = sequence

#getting unique token list
unique_tokens = set(tokensList)

#data variables
vocab_size = len(unique_tokens)
seq_length = max_sequence

##printing data information
print('Source Data Information:')
print("The number of source tokens are ", len(tokensList))
print("The number of unique source tokens are ", len(unique_tokens))
print("Max source sequence length of source ",max_sequence)
print("Source Vocabular: ", unique_tokens)

## Train and Test Data

In [23]:
'''
Splitting data based on Cultural data and Musical Feature data
'''

train_CulturalData = train_Data["Text"]
test_CulturalData = test_Data["Text"]

train_MusicalFeatureData = train_Data[1:16]
test_MusicalFeatureData = test_Data[1:16]

max_features = len(unique_tokens)
tokenizer = Tokenizer(num_words=max_sequence)
tokenizer.fit_on_texts(list(all_CulturalData))


list_tokenized_train = tokenizer.texts_to_sequences(train_CulturalData)
X_train = pad_sequences(list_tokenized_train, maxlen=vocab_size)

list_tokenized_test = tokenizer.texts_to_sequences(test_CulturalData)
X_test = pad_sequences(list_tokenized_test, maxlen=vocab_size)

y = train_Data["Sales"].values

# Models

## Cultural Data

### LSTM

In [24]:
'''
LSTM model for cultural data
'''
class Model_LSTM_CulturalData():
    def __new__(self):
      embed_size = 128
      model = Sequential()
      model.add(Embedding(max_features, embed_size))
      model.add(LSTM(256, return_sequences=True))
      model.add(Dense(256, activation='relu'))
      model.add(Dropout(0.1))
      model.add(Dense(1, activation='softmax'))

      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
      return model
    
model_LSTM_CD = Model_LSTM_CulturalData()

In [ ]:
#printing model summary
print(model_LSTM_CD.summary())

In [ ]:
#training model
model_LSTM_CD.fit(X_train, y, batch_size=16, epochs=200)

In [ ]:
#Getting predictions
LSTM_CulturalData_Predictions = model_LSTM_CD.predict(X_test)

### RNN 

In [ ]:
'''
RNN model for cultural data
'''
class Model_RNN_CulturalData():
    def __new__(self):
      embed_size = 128
      model = Sequential()
      model.add(Embedding(max_features, embed_size))
      model.add(SimpleRNN(256, return_sequences=True))
      model.add(Dense(256, activation='relu'))
      model.add(Dropout(0.1))
      model.add(Dense(len(y), activation='softmax'))

      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
      return model
    
model_RNN_CD = Model_RNN_CulturalData()

In [ ]:
#printing model summary
print(model_RNN_CD.summary())

In [ ]:
#training model
model_RNN_CD.fit(X_train, y, batch_size=16, epochs=200)

In [ ]:
#Getting predictions
RNN_CulturalData_Predictions = model_RNN_CD.predict(X_test)

### CNN 



In [ ]:
'''
CNN model for cultural data
'''
class Model_CNN_CulturalData():
    def __new__(self):
      embed_size = 128
      kernel_size <- 8
      hidden_dims <- 256
      model = Sequential()
      model.add(Embedding(max_features, embed_size))
      model.add(Conv1D(64, kernel_size, padding = "valid", activation = "relu", strides = 1))
      model.add(GlobalMaxPool1D(2)))
      model.add(Dense(hidden_dims))
      model.add(Dropout(0.1))
      model.add(Dense(1, activation='softmax'))

      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
      return model
    
model_CNN_CD = Model_CNN_CulturalData()

In [ ]:
#printing model summary
print(model_CNN_CD.summary())

In [ ]:
#training model
model_CNN_CD.fit(train_, y, batch_size=16, epochs=200)

In [ ]:
#Getting predictions
CNN_CulturalData_Predictions = model_CNN_CD.predict(X_test)

## Musical Data

### LSTM 

In [ ]:
'''
LSTM model for musical data
'''
class Model_LSTM_MusicalData():
    def __new__(self):
      embed_size = 128
      model = Sequential()
      model.add(Embedding(max_features, embed_size))
      model.add(LSTM(256, return_sequences=True))
      model.add(Dense(256 activation='relu'))
      model.add(Dropout(0.1))
      model.add(Dense(1, activation='softmax'))

      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
      return model
    
model_LSTM_MD = Model_LSTM_MusicalData()

In [ ]:
#printing model summary
print(model_LSTM_MD.summary())

In [ ]:
#Training Model
model_LSTM_MD.fit(train_MusicalFeatureData, y, batch_size=16, epochs=200)

In [ ]:
#Model Predictions
LSTM_MusicalData_Predictions = model_LSTM_MD.predict(test_MusicalFeatureData)

### RNN 

In [ ]:
'''
RNN model for musical data
'''
class Model_RNN_MusicalData():
    def __new__(self):
      embed_size = 128
      model = Sequential()
      model.add(Embedding(max_features, embed_size))
      model.add(SimpleRNN(256, return_sequences=True))
      model.add(Dense(256, activation='relu'))
      model.add(Dropout(0.1))
      model.add(Dense(1, activation='softmax'))

      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
      return model
    
model_RNN_MD = Model_RNN_MusicalData()

In [ ]:
#printing model summary
print(model_RNN_MD.summary())

In [ ]:
#Training Model
model_RNN_MD.fit(train_MusicalFeatureData, y, batch_size=16, epochs=200)

In [ ]:
#Model Predictions
RNN_MusicalData_Predictions = model_RNN_MD.predict(test_MusicalFeatureData)

### CNN 



In [ ]:
'''
CNN model for musical data
'''
class Model_CNN_MusicalData():
    def __new__(self):
      embed_size = 128
      kernel_size <- 8
      hidden_dims <- 256
      model = Sequential()
      model.add(Embedding(max_features, embed_size))
      model.add(Conv1D(256, kernel_size, padding = "valid", activation = "relu", strides = 1))
      model.add(GlobalMaxPool1D(2)))
      model.add(Dense(hidden_dims))
      model.add(Dropout(0.1))
      model.add(Dense(1, activation='softmax'))

      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
      return model
    
model_CNN_MD = Model_CNN_MusicalData()

In [ ]:
#printing model summary
print(model_CNN_CD.summary())

In [ ]:
#Training Model
model_CNN_MD.fit(train_MusicalFeatureData, y, batch_size=16, epochs=200)

In [ ]:
#Getting Model Predictions
CNN_MusicalData_Predictions = model_CNN_MD.predict(test_MusicalFeatureData)

## Combined Models

In [ ]:
#Combined data for testing
test_MusicalFeatureData['CulturalData'] = X_test
test_CombinedData = test_MusicalFeatureData

### LSTM

In [ ]:
'''
LSTM model for combined data
'''
class Model_LSTM_CombinedData():
    def __new__(self):
      model = concatenate([model_LSTM_MD, model_LSTM_CD],axis=-1)
      model.add(LSTM(256, return_sequences=False))
      model.add(Dense(1, activation='softmax'))
      
      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


model_LSTM_CombinedD = Model_LSTM_CombinedData()

In [ ]:
print(model_LSTM_CombinedD.summary())

In [ ]:
LSTM_CombinedData_Predictions = model_LSTM_CombinedD.predict(test_CombinedData )

### RNN

In [ ]:
'''
RNN model for combined data
'''
class Model_RNN_CombinedData():
    def __new__(self):
      model = concatenate([model_RNN_MD, model_RNN_CD],axis=-1)
      model.add(SimpleRNN(256))
      model.add(Dense(1, activation='softmax'))
      
      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


model_RNN_CombinedD = Model_RNN_CombinedData()

In [ ]:
print(model_RNN_CombinedD.summary())

In [ ]:
RNN_CombinedData_Predictions = model_RNN_CombinedD.predict(test_CombinedData )

### CNN

In [ ]:
'''
CNN model for combined data
'''

class Model_CNN_CombinedData():
    def __new__(self):
      kernel_size <- 8
      model = concatenate([model_CNN_MD, model_CNN_CD],axis=-1)
      model.add(Conv1D(128, kernel_size, padding = "valid", activation = "relu", strides = 1))
      model.add(GlobalMaxPool1D(2)))
      model.add(Dense(1, activation='softmax'))
      
      model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


model_CNN_CombinedD = Model_CNN_CombinedData()

In [ ]:
print(model_CNN_CombinedD.summary())

In [ ]:
CNN_CombinedData_Predictions = model_CNN_CombinedD.predict(test_CombinedData )

# Results

In [ ]:
'''
Getting actual sales values of test data
'''
actualSales = test_Data['Sales'].values
size_test = len(test_Data)
acceptable_range = 0.05

In [ ]:
#Creating dataframe of predictions
salesDF = pd.DataFrame([actualSales , LSTM_CulturalData_Predictions , LSTM_MusicalData_Predictions , LSTM_CombinedData_Predictions ,
                        RNN_CulturalData_Predictions , RNN_MusicalData_Predictions, RNN_CombinedData_Predictions,
                        CNN_CulturalData_Predictions , CNN_MusicalData_Predictions, CNN_CombinedData_Predictions],
                       index = ['Actual Sales' , 'LSTM CD Sales' , 'LSTM MD Sales' , 'LSTM C Sales',
                                'RNN CD Sales' , 'RNN MD Sales' , 'RNN C Sales',
                                'CNN CD Sales' , 'CNN MD Sales' , 'CNN C Sales'])

In [ ]:
salesDF.to_excel('salesPredictions.xlsx')

## Error Calculation

In [ ]:
'''
Calculating RMSE and MAE for each of the Models
'''
MAE_values = []
RMSE_values = []

In [ ]:
#Variables for each model
n = len(actualSales)
LSTM_CulturalData_sumError = 0  
LSTM_CulturalData_sumAbsError = 0 

LSTM_MusicalData_sumError = 0
LSTM_MusicalData_sumAbsError = 0

LSTM_CombinedData_sumError = 0
LSTM_CombinedData_sumAbsError = 0

###

RNN_CulturalData_sumError = 0
RNN_CulturalData_sumAbsError = 0

RNN_MusicalData_sumError = 0
RNN_MusicalData_sumAbsError = 0

RNN_CombinedData_sumError = 0
RNN_CombinedData_sumAbsError = 0

###

CNN_CulturalData_sumError = 0
CNN_CulturalData_sumAbsError = 0

CNN_MusicalData_sumError = 0
CNN_MusicalData_sumAbsError = 0

CNN_CombinedData_sumError = 0
CNN_CombinedData_sumAbsError = 0


for s in range(n):
  #Calculations of sum absolute error and sum square error for each model
  LSTM_CulturalData_Error = actualSales[s] - LSTM_CulturalData_Predictions[s] 
  LSTM_CulturalData_AbsError = abs(LSTM_CulturalData_Error) 
  LSTM_CulturalData_SqrError = LSTM_CulturalData_Error ** 2

  LSTM_CulturalData_sumError +=  LSTM_CulturalData_SqrError 
  LSTM_CulturalData_sumAbsError += LSTM_CulturalData_AbsError

  #
  LSTM_MusicalData_Error = actualSales[s] - LSTM_MusicalData_Predictions[s] 
  LSTM_MusicalData_AbsError = abs(LSTM_MusicalData_Error)
  LSTM_MusicalData_SqrError = LSTM_MusicalData_Error ** 2

  LSTM_MusicalData_sumError +=  LSTM_MusicalData_SqrError 
  LSTM_MusicalData_sumAbsError += LSTM_MusicalData_AbsError

  #
  LSTM_CombinedData_Error = actualSales[s] - LSTM_CombinedData_Predictions[s]
  LSTM_CombinedData_AbsError = abs(LSTM_CombinedData_Error)
  LSTM_CombinedData_SqrError = LSTM_CombinedData_Error ** 2

  LSTM_CombinedData_sumError +=  LSTM_CombinedData_SqrError 
  LSTM_CombinedData_sumAbsError += LSTM_CombinedData_AbsError 


  ###
  RNN_CulturalData_Error = actualSales[s] - RNN_CulturalData_Predictions[s]
  RNN_CulturalData_AbsError = abs(RNN_CulturalData_Error)
  RNN_CulturalData_SqrError = RNN_CulturalData_Error ** 2

  RNN_CulturalData_sumError +=  RNNCulturalData_SqrError 
  RNN_CulturalData_sumAbsError += RNN_CulturalData_AbsError

  #
  RNN_MusicalData_Error = actualSales[s] - RNN_MusicalData_Predictions[s]
  RNN_MusicalData_AbsError = abs(RNN_MusicalData_Error)
  RNN_MusicalData_SqrError = RNN_MusicalData_Error ** 2

  RNN_MusicalData_sumError +=  RNN_MusicalData_SqrError 
  RNN_MusicalData_sumAbsError += RNN_MusicalData_AbsError

  #
  RNN_CombinedData_Error = actualSales[s] - RNN_CombinedData_Predictions[s]
  RNN_CombinedData_AbsError = abs(RNN_CombinedData_Error)
  RNN_CombinedData_SqrError = RNN_CombinedData_Error ** 2

  RNN_CombinedData_sumError +=  RNN_CombinedData_SqrError 
  RNN_CombinedData_sumAbsError += RNN_CombinedData_AbsError 


  ###
  CNN_CulturalData_Error = actualSales[s] - CNN_CulturalData_Predictions[s]
  CNN_CulturalData_AbsError = abs(CNN_CulturalData_Error)
  CNN_CulturalData_SqrError = CNN_CulturalData_Error ** 2

  CNN_CulturalData_sumError +=  CNNCulturalData_SqrError 
  CNN_CulturalData_sumAbsError += CNN_CulturalData_AbsError

  #
  CNN_MusicalData_Error = actualSales[s] - CNN_MusicalData_Predictions[s]
  CNN_MusicalData_AbsError = abs(CNN_MusicaldData_Error)
  CNN_MusicalData_SqrError = CNN_MusicalData_Error ** 2

  CNN_MusicalData_sumError +=  CNN_MusicalData_SqrError 
  CNN_MusicalData_sumAbsError += CNN_MusicalData_AbsError

  #
  CNN_CombinedData_Error = actualSales[s] - CNN_CombinedData_Predictions[s]
  CNN_CombinedData_AbsError = abs(CNN_CombinedData_Error)
  CNN_CombinedData_SqrError = CNN_CombinedData_Error ** 2

  CNN_CombinedData_sumError +=  CNN_CombinedData_SqrError 
  CNN_CombinedData_sumAbsError += CNN_CombinedData_AbsError 


##Calculations of MAE and RMSE for each model

LSTM_CulturalData_MSE = LSTM_CulturalData_sumError/n
LSTM_CulturalData_RMSE = math.sqrt(LSTM_CulturalData_MSE) 
LSTM_CulturalData_MAE = LSTM_CulturalData_sumAbsError/n
#
LSTM_MusicalData_MSE = LSTM_MusicalData_sumError/n
LSTM_MusicalData_RMSE = math.sqrt(LSTM_MusicalData_MSE) 
LSTM_MusicalData_MAE = LSTM_MusicalData_sumAbsError/n
#
LSTM_CombinedData_MSE = LSTM_CombinedData_sumError/n
LSTM_CombinedData_RMSE = math.sqrt(LSTM_CombinedData_MSE) 
LSTM_CombinedData_MAE = LSTM_CombinedData_sumAbsError/n

###
RNN_CulturalData_MSE = RNN_CulturalData_sumError/n
RNN_CulturalData_RMSE = math.sqrt(RNN_CulturalData_MSE) 
RNN_CulturalData_MAE = RNN_CulturalData_sumAbsError/n
#
RNN_MusicalData_MSE = RNN_MusicalData_sumError/n
RNN_MusicalData_RMSE = math.sqrt(RNN_MusicalData_MSE) 
RNN_MusicalData_MAE = RNN_MusicalData_sumAbsError/n 
#
RNN_CombinedData_MSE = RNN_CombinedData_sumError/n
RNN_CombinedData_RMSE = math.sqrt(RNN_CombinedData_MSE) 
RNN_CombinedData_MAE = RNN_CombinedData_sumAbsError/n 

###
CNN_CulturalData_MSE = CNN_CulturalData_sumError/n
CNN_CulturalData_RMSE = math.sqrt(CNN_CulturalData_MSE) 
CNN_CulturalData_MAE = CNN_CulturalData_sumAbsError/n
#
CNN_MusicalData_MSE = CNN_MusicalData_sumError/n
CNN_MusicalData_RMSE = math.sqrt(CNN_MusicalData_MSE) 
CNN_MusicalData_MAE = CNN_MusicalData_sumAbsError/n 
#
CNN_CombinedData_MSE = CNN_CombinedData_sumError/n
CNN_CombinedData_RMSE = math.sqrt(CNN_CombinedData_MSE) 
CNN_CombinedData_MAE = CNN_CombinedData_sumAbsError/n 

#Adding MAE Values List
MAE_values.append(LSTM_CulturalData_MAE)
MAE_values.append(LSTM_MusicalData_MAE)
MAE_values.append(LSTM_CombinedData_MAE)

MAE_values.append(RNN_CulturalData_MAE)
MAE_values.append(RNN_MusicalData_MAE)
MAE_values.append(RNN_CombinedData_MAE)

MAE_values.append(CNN_CulturalData_MAE)
MAE_values.append(CNN_MusicalData_MAE)
MAE_values.append(cNN_CombinedData_MAE)

###

#Adding RMSE Values List
RMSE_values.append(LSTM_CulturalData_RMSE)
RMSE_values.append(LSTM_MusicalData_RMSE)
RMSE_values.append(LSTM_CombinedData_RMSE)

RMSE_values.append(RNN_CulturalData_RMSE)
RMSE_values.append(RNN_MusicalData_RMSE)
RMSE_values.append(RNN_CombinedData_RMSE)

RMSE_values.append(CNN_CulturalData_RMSE)
RMSE_values.append(CNN_MusicalData_RMSE)
RMSE_values.append(CNN_CombinedData_RMSE)

In [ ]:
#Creating dataframe from MAE and RMSE values
analysisDF = pd.DataFrame([MAE_values,RMSE_values],
                          index = ['MAE' , 'RMSE'],
                          columns = ['LSTM CD' , 'LSTM MD' , 'LSTM C',
                                      'RNN CD' , 'RNN MD' , 'RNN C',
                                      'CNN CD' , 'CNN MD' , 'CNN C'])

In [ ]:
analysisDF.to_excel('errorAnalysisDF.xlsx')

## Comparing sales predictions to actual sales

In [ ]:
'''
Comparing predicted sales value to actual sales value 
  -If predicted value is within 5% of actual value the predicted value is set to be predicted accuratly 
'''

def performance_results(predictions):
    score = 0
    for prediction in range(len(predictions)):
      actualSales = actualSales[prediction]
      predictedSales = predictions[prediction]
      difference = abs(actualSales - predictedSales)
      difference_as_percent = difference/actualSales
      if difference_as_percent < acceptable_range:
          score += 1
          
    performance_score = score/size_test
    return performance_score

In [ ]:
'''
Running performance function on predictinos of each model for cultural data
'''

CulturalData_Performance = []
CulturalData_Performance.append(performance_results(LSTM_CulturalData_Predictions))
CulturalData_Performance.append(performance_results(RNN_CulturalData_Predictions))
CulturalData_Performance.append(performance_results(CNN_CulturalData_Predictions))

In [ ]:
'''
Running performance function on predictinos of each model for musical data
'''


MusicalData_Performance = []
MusicalData_Performance.append(performance_results(LSTM_MusicalData_Predictions))
MusicalData_Performance.append(performance_results(RNN_MusicalData_Predictions))
MusicalData_Performance.append(performance_results(CNN_MusicalData_Predictions))

In [ ]:
'''
Running performance function on predictinos of each model for the combined data
'''


CombinedData_Performance = []
CombinedData_Performance.append(performance_results(LSTM_CombinedData_Predictions))
CombinedData_Performance.append(performance_results(RNN_CombinedData_Predictions))
CombinedData_Performance.append(performance_results(CNN_CombinedData_Predictions))

In [ ]:
'''
Creating table of performance results and writing to excel sheet
'''

results = ['LSTM','RNN','CNN']
results['Cultural Data'] = CulturalData_Performance
results['Musical Data'] = MusicalData_Performance
results['Combined Data'] = CombinedData_Performance

In [ ]:
results.to_excel('PerformanceResults.xlsx',index=False)